In [7]:
import pandas as pd
import numpy as np
import ssl

ssl._create_default_https_context = ssl._create_unverified_context # Para que la URL del archivo cvs funcione en mac
ruta = "https://raw.githubusercontent.com/lbtriana/Proyecto_Bicicletas/main/SeoulBikeData_utf8.csv" #ruta desde url
df = pd.read_csv(ruta)
df['fecha'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['day'] = df['fecha'].dt.day
df['month'] = df['fecha'].dt.month
df['year'] = df['fecha'].dt.year
df['day_of_week'] = df['fecha'].dt.dayofweek
df['functioning_day'] = df['Functioning Day'].replace({'Yes': 1, 'No': 0})
df['is_holiday'] = df['Holiday'].replace({'Holiday': 1, 'No Holiday': 0})
pd.set_option('future.no_silent_downcasting', True) # Esta linea es para evitar un warning de perder demasiada info al hacer downcasting
df['Hour_PM'] = np.where(df['Hour'] >= 12, 1, 0) #1 si es hora en la tarde 
df['Hour_lab'] = np.where((df['Hour'] >= 8) & (df['Hour'] <= 17), 1, 0) #1 si es horario laboral

df.info()

var_continuas = [
    'Hour', 
    'Temperature(C)', 
    'Humidity(%)', 
    'Wind speed (m/s)', 
    'Visibility (10m)', 
    'Dew point temperature(C)', 
    'Solar Radiation (MJ/m2)', 
    'Rainfall(mm)', 
    'Snowfall (cm)', 
    'day', 
    'month', 
    'year', 
    'day_of_week',
    'fecha',
    'functioning_day', 
    'is_holiday', 
    'Rented Bike Count'
]

var_categoricas = [
    'Seasons', 
    'Hour_PM', 
    'Hour_lab'
]

continuas=df[var_continuas]
categoricas=df[var_categoricas]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      8760 non-null   object        
 1   Rented Bike Count         8760 non-null   int64         
 2   Hour                      8760 non-null   int64         
 3   Temperature(C)            8760 non-null   float64       
 4   Humidity(%)               8760 non-null   int64         
 5   Wind speed (m/s)          8760 non-null   float64       
 6   Visibility (10m)          8760 non-null   int64         
 7   Dew point temperature(C)  8760 non-null   float64       
 8   Solar Radiation (MJ/m2)   8760 non-null   float64       
 9   Rainfall(mm)              8760 non-null   float64       
 10  Snowfall (cm)             8760 non-null   float64       
 11  Seasons                   8760 non-null   object        
 12  Holiday             

In [8]:
df_cat = pd.get_dummies(categoricas, dtype="float64")

df_cat.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Hour_PM         8760 non-null   int64  
 1   Hour_lab        8760 non-null   int64  
 2   Seasons_Autumn  8760 non-null   float64
 3   Seasons_Spring  8760 non-null   float64
 4   Seasons_Summer  8760 non-null   float64
 5   Seasons_Winter  8760 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 410.8 KB


In [9]:
df = pd.concat([continuas, df_cat], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Hour                      8760 non-null   int64         
 1   Temperature(C)            8760 non-null   float64       
 2   Humidity(%)               8760 non-null   int64         
 3   Wind speed (m/s)          8760 non-null   float64       
 4   Visibility (10m)          8760 non-null   int64         
 5   Dew point temperature(C)  8760 non-null   float64       
 6   Solar Radiation (MJ/m2)   8760 non-null   float64       
 7   Rainfall(mm)              8760 non-null   float64       
 8   Snowfall (cm)             8760 non-null   float64       
 9   day                       8760 non-null   int32         
 10  month                     8760 non-null   int32         
 11  year                      8760 non-null   int32         
 12  day_of_week         

In [17]:
features = [
    'Hour', 
    'Temperature(C)', 
    'Humidity(%)', 
    'Wind speed (m/s)', 
    'Visibility (10m)',     
    'Dew point temperature(C)', 
    'Solar Radiation (MJ/m2)', 
    'Rainfall(mm)', 
    'Snowfall (cm)', 
    'Seasons_Autumn', 
    'Seasons_Winter',
    'Seasons_Spring',
    'Seasons_Summer',
    'day', 
    'month', 
    'year', 
    'day_of_week', 
    'functioning_day', 
    'is_holiday', 
    'Hour_PM', 
    'Hour_lab'
]

X = df[features]

Y = df['Rented Bike Count']

from sklearn.model_selection import train_test_split 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

from sklearn.linear_model import LinearRegression

# crear el objeto del modelo
linreg = LinearRegression()

# ajustar los parámetros del modelo usando los datos de entrenamiento
linreg.fit(X_train, Y_train)



LinearRegression()

In [18]:
# imprimir coeficientes
print(linreg.intercept_)
print(linreg.coef_)


1188235.9053979341
[ 3.03586113e+01  1.45354410e+01 -1.20535915e+01  1.54933661e+01
 -1.04437978e-03  1.31225832e+01 -7.22429439e+01 -5.23775535e+01
  3.50453608e+01  3.79470559e+02 -4.19189814e+02 -4.00315874e+01
  7.97508420e+01 -4.89452379e-01 -4.69827737e+01 -5.88678386e+02
 -1.16377051e+01  9.22526702e+02 -1.41173778e+02 -5.29423502e+01
 -1.14356413e+01]


In [20]:
# coeficientes con nombre de las características
list(zip(features, linreg.coef_))


[('Hour', np.float64(30.35861132638185)),
 ('Temperature(C)', np.float64(14.535441026830439)),
 ('Humidity(%)', np.float64(-12.053591487233616)),
 ('Wind speed (m/s)', np.float64(15.493366124719078)),
 ('Visibility (10m)', np.float64(-0.0010443797846477704)),
 ('Dew point temperature(C)', np.float64(13.12258323590411)),
 ('Solar Radiation (MJ/m2)', np.float64(-72.24294391004887)),
 ('Rainfall(mm)', np.float64(-52.377553465713525)),
 ('Snowfall (cm)', np.float64(35.04536082478358)),
 ('Seasons_Autumn', np.float64(379.47055938715016)),
 ('Seasons_Winter', np.float64(-419.18981397147076)),
 ('Seasons_Spring', np.float64(-40.03158743259836)),
 ('Seasons_Summer', np.float64(79.75084201691911)),
 ('day', np.float64(-0.4894523791300794)),
 ('month', np.float64(-46.98277368938033)),
 ('year', np.float64(-588.6783860273306)),
 ('day_of_week', np.float64(-11.63770509380178)),
 ('functioning_day', np.float64(922.5267020428017)),
 ('is_holiday', np.float64(-141.17377839980966)),
 ('Hour_PM', np.fl

In [21]:
y_pred = linreg.predict(X_test)

In [22]:
from sklearn import metrics

In [24]:
import numpy as np

# mean absolute error
MAE = metrics.mean_absolute_error(Y_test, y_pred)

# mean squared error
MSE = metrics.mean_squared_error(Y_test, y_pred)

# root mean squared error
RMSE = np.sqrt(MSE)

In [25]:
print("MAE: ", MAE)
print("MSE: ", MSE)
print("RMSE: ", RMSE)

MAE:  319.6044187210984
MSE:  183475.2359410682
RMSE:  428.3400937818782


In [27]:
from sklearn.model_selection import cross_val_score

# usar MSE - error cuadrático medio
scores = cross_val_score(linreg, X, Y, cv=5, scoring='neg_mean_squared_error')
mse_scores = - scores
print(mse_scores)

[237581.91692081 357285.57982567 443348.3364232  478862.62482427
 222781.10971963]


In [28]:
# calcular RMSE
rmse_scores = np.sqrt(mse_scores)
print(rmse_scores)

[487.42375498 597.7337031  665.84407816 691.99900638 471.99693825]


In [29]:
# RMSE promedio a través de todos los grupos
print(rmse_scores.mean())

582.9994961749734


In [32]:
import statsmodels.api as sm


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

# agregar constante explíticamente
X_train = sm.add_constant(X_train)

# regresión usando mínimos cuadrados ordinarios (ordinary least squares - OLS) 
model = sm.OLS(Y_train, X_train).fit()

# resumen de resultados
print(model.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).